In [125]:
blob_relative_path = "immigration_yearbook_excel_files/2021/2022_0920_plcy_refugees_and_asylees_fy2021_tables.zip/"

StatementMeta(smallpool33, 366, 126, Finished, Available)

In [126]:
from notebookutils import mssparkutils  
from pyspark.sql import SparkSession 
from pyspark.sql.types import * 
import re
import pandas as pd
import numpy as np
from azure.storage.blob import BlobServiceClient
from pyspark.sql.functions import *
# Azure storage access info 
blob_account_name = 'usafactsbronze' # replace with your blob name 
blob_container_name = 'bronze' # replace with your container name 
linked_service_name = 'bronze' # replace with your linked service name 

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 

# Allow SPARK to access from Blob remotely 
wasbs_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path) 
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 

blob_service_client = BlobServiceClient(account_url=f'https://{blob_account_name}.blob.core.windows.net/', credential=blob_sas_token)
container_client = blob_service_client.get_container_client(blob_container_name)


StatementMeta(smallpool33, 366, 127, Finished, Available)

In [127]:
pip install xlrd

StatementMeta(smallpool33, 366, 128, Finished, Available)

Note: you may need to restart the kernel to use updated packages.


In [128]:
# Azure storage access info 
blob_account_name = 'usafactssilver' # replace with your blob name 
blob_container_name = 'silver' # replace with your container name 
linked_service_name = 'silver' # replace with your linked service name 

blob_sas_token = mssparkutils.credentials.getConnectionStringOrCreds(linked_service_name) 

# Allow SPARK to access from Blob remotely 
target_path = 'wasbs://%s@%s.blob.core.windows.net/%s' % (blob_container_name, blob_account_name, blob_relative_path) 
spark.conf.set('fs.azure.sas.%s.%s.blob.core.windows.net' % (blob_container_name, blob_account_name), blob_sas_token) 

StatementMeta(smallpool33, 366, 129, Finished, Available)

In [129]:
def get_file_paths(dir_path):
    file_paths= []
    files = mssparkutils.fs.ls(dir_path)
    for file in files:
        allowed_extensions = ['.xls', '.xlsx']
        if file.isDir :
            file_paths.extend(get_file_paths(file.path))
        if any(file.path.endswith(ext) for ext in allowed_extensions):
            file_paths.append(file.path)

    return file_paths

StatementMeta(smallpool33, 366, 130, Finished, Available)

In [130]:
all_file_paths = get_file_paths(wasbs_path)

StatementMeta(smallpool33, 366, 131, Finished, Available)

In [131]:
def concat_xls_row(data,none_count):
                
    concat_list1 = []
    concat_list = []
    if none_count == 1:
        iter_range = 2
    else:
        iter_range = none_count
    for index in range(iter_range):
        if (none_count == 1 and index == 0) or index+1 < none_count :
            temp_value = None
            temp_list =[]

            for value in data[index]:
                if value is None:
                    value = temp_value
                else:
                    temp_value = value
                temp_list.append(value)
            cleaned_row = [value for value in data[index] if value is not None]
            if  index == 0 and len(cleaned_row) == 1:
                temp_list = [None] + (cleaned_row *(len(data[0])-1))
            if len(concat_list1) >= 1:
                concat_list1 = [(x if x is not None else ' ') +' '+ (y if y is not None else ' ') for x, y in zip(concat_list1,temp_list)]
            else:
                concat_list1 =  temp_list
        else:
            concat_list = [(x if x is not None else ' ') +' '+ (y if y is not None else ' ') for x, y in zip(concat_list1,data[index])]
            data[index] = concat_list
            
            if none_count == 1 :
                data = data[index:]
            else:
                data = data[none_count-1:]
    return data

StatementMeta(smallpool33, 366, 132, Finished, Available)

In [132]:
def create_df (data,header,footer): 
    cleaned_columns = [re.sub(r'[^a-zA-Z0-9]', "_", value.strip().replace('.0','')) if value is not None else ('col_'+str(ind)) for ind,value in enumerate(data[0])]
    columns = []

    if len(cleaned_columns) !=  len(list(set(cleaned_columns))):
        for ind,column in enumerate(cleaned_columns) :
            if column not in columns :
                columns.append(column)
            else:
                columns.append(column+'_'+str(ind))
    else:
        columns = cleaned_columns
    schema = StructType([StructField(name, StringType(), True) for name in columns])
    
    df = spark.createDataFrame(data[1:], schema)
    empty_columns = [column for column in df.columns if df.filter(df[column].isNull()).count() == df.count() ]
    df = df.drop(*empty_columns)

    if len(df.columns) != len(list(set(df.columns))):
        columns = []
        for ind,col in enumerate(df.columns):
            columns.append('col_'+str(ind))
        df = spark.createDataFrame(data,columns)
    if header:
        header = '. '.join(header)
        df = df.withColumn("Header",lit(header))
    if footer:
        footer ='. '.join(footer)
        df = df.withColumn("Footer",lit(footer))
    return df

StatementMeta(smallpool33, 366, 133, Finished, Available)

In [133]:
def seperate_tables(raw_data):
    datas = []
    temp_header = []
    temp_data = []
    is_header = False
    for ind, row in enumerate(raw_data):
        
        row = [None if value =='nan' or value.strip() == '' else value.replace('……...', '    ').replace('…...', '   ').replace('...',' ').replace('…..',' ').replace('….', '  ').replace('..',' ').replace('  .','').replace('….','').replace('…','') for value in row ]
        cleaned_row = [value for value in row if value is not None]

        if row[0] is None and len(cleaned_row)==2:
            row[2:] = [None] * (len(row) - 2)
        if len(cleaned_row) < 1:
            continue
        
        if ((len(cleaned_row) == 1 and row[0] is None) 
            or (len(raw_data[ind-1])==raw_data[ind-1].count('nan') and len(raw_data[ind-2])==raw_data[ind-2].count('nan') and row[0] is None)
            or (len(raw_data[ind-1])==raw_data[ind-1].count('nan') and row[0] is None and len(list(set(cleaned_row))) == 1 )
            or (len(raw_data[ind-1])==raw_data[ind-1].count('nan') and len(cleaned_row) != 1 and raw_data[ind].count('nan')>1 and row[0] is not None and row[2] is None)
            or (len(raw_data[ind-1])==raw_data[ind-1].count('nan') and row[0] is not None  and row[0].startswith('Revised, based on Vintage'))):
            if is_header :
                datas.append(temp_header+temp_data)
                temp_data = []
            is_header = True
        if len(cleaned_row) == 1 and is_header and cleaned_row[0].strip().startswith('Table'):
            is_header = False
            datas.append(temp_header+temp_data)
            
            temp_data = []
            temp_header = []

        if is_header :  
            temp_data.append(row)

        if len(temp_data) < 1 and not is_header :
            temp_header.append(row)
        if ind+1 == len(raw_data):
            datas.append(temp_header+temp_data)
    return datas

StatementMeta(smallpool33, 366, 134, Finished, Available)

In [139]:
def filtered_data(clean_data):
    # print(clean_data)
    datas = seperate_tables(clean_data)
    dfs =[]
    for df_data in datas :
     
        data = []
        temp_footer = ''
        header= []
        footer = []
        for ind , row in enumerate(df_data):
            cleaned_row = [value for value in row if value is not None]
            if len(cleaned_row ) > 1 or (len(cleaned_row) == 1 and row[0] is None):
                if temp_footer :
                    if row[0] is None:
                        row[0] = ''
                    row[0] = temp_footer +'  ' + row[0]
                data.append(row)
            elif len(data) < 1 :
                header += cleaned_row 
            else:
                if len(cleaned_row) !=0:
                    temp_footer = cleaned_row[0]
                    
                    row = [None if value is None or value.strip() == '' else value for value in df_data[ind-1] ]
                    prev_clean_row = [value for value in row if value is not None]
                    if len(prev_clean_row ) < 2   :
                        footer += cleaned_row
                    else:
                        footer = cleaned_row

        none_count = 0 
        none_exclude = 0
        # print(data)
        for value in data :
            if ((value.count(None) > 1 and None in value[:3]) or (none_count <= 1 and value.count(None) == 1  and value[-1] is not None)  or value[0] is None 
                or all(x.isalpha() if x is not None else True for x in value[1:]) or (value.count(None) == 2 and value[0] is None and value[-1] is None)):
                 
                none_count += 1
            else:
                break 

        data = concat_xls_row(data,none_count-1)
        # print(data)
        if len(data) !=0:
            level_list = []
            level_dict = {}
            clean_data = [data[0]]
            for ind,row in enumerate(data[1:]):
                s = row[0]
                if row[0]!=None:
                    current_key = len(s)-len(s.lstrip())
                    if current_key == 0 :
                        level_dict = {}
                    keys_to_delete = [key for key in level_dict.keys() if key > current_key]
                    for key in keys_to_delete:
                        del level_dict[key]
                    level_dict[current_key] = s.strip()
                    row[0] = ' '.join(level_dict.values())
                    cleaned_row = [value for value in row if value is not None]
                    if len(cleaned_row) < 2:
                        pass
                    else:
                        clean_data.append(row)

        if len(clean_data)>0:
            df = create_df(clean_data,header,footer)
            # display(df)      
            dfs.append(df)
    return dfs

StatementMeta(smallpool33, 366, 140, Finished, Available)

In [135]:
import warnings

# Ignore the specific FutureWarning related to iteritems
warnings.filterwarnings("ignore", category=FutureWarning, module="pyspark")

StatementMeta(smallpool33, 366, 136, Finished, Available)

In [140]:
from urllib.parse import quote
bad_records = []
for text_path in all_file_paths:
    try:
        file_path = text_path.split('.net/')[-1]
        file_path = quote(file_path, safe="/:")
        file_location = target_path + text_path.split(wasbs_path)[-1]
        file_location='.'.join(file_location.split('.')[:-1])
        file_name=file_location.split('/')[-1]
        file_location=file_location+'/'+file_name
        link = f'https://usafactsbronze.blob.core.windows.net/bronze/{file_path}'
        df = pd.read_excel(link,header = None)
        
        df = df.applymap(lambda x: np.nan if isinstance(x, str) and x.strip()=='' else x)
        df = df.dropna(axis=1, how='all') 
        df = df.astype(str)
        
        df = spark.createDataFrame(df)

        first_row = [value for value in df.first()]
        if 'nan' in first_row :
            raw_data = df.collect()
            dfs= filtered_data(raw_data)
            # print(dfs)
            footer_value = []
            if len(dfs) > 1:
                last_df = dfs[-1]  
                footer_value = last_df.select('footer').collect()[0][0]  
                first_df=dfs[0]
                Header_value = first_df.select('Header').collect()[0][0]  
                for ind,df in enumerate(dfs):
                    df = df.withColumn('Header',lit(Header_value)).withColumn('footer', lit(footer_value)).withColumnRenamed('col_0', "Description").withColumnRenamed('', "Description")
                    display(df)
                    # df.write.format('delta').mode('overwrite').option("overwriteSchema", "true").option("path",file_location+'_'+str(ind)).save()
            else:
                print(len(dfs))
                display(dfs[0])
                # dfs[0].write.format('delta').mode('overwrite').option("overwriteSchema", "true").option("path",file_location).save()
        print(file_location,'uploaded sucessfully')

    except BaseException as e :
        bad_records1.append((text_path,file_location.split('/')[-1],e))
        print(e,file_location)

StatementMeta(smallpool33, 366, 141, Finished, Available)

1


SynapseWidget(Synapse.DataFrame, a816ae00-ec39-4c8d-b484-8a7f7d492e70)

wasbs://silver@usafactssilver.blob.core.windows.net/immigration_yearbook_excel_files/2021/2022_0920_plcy_refugees_and_asylees_fy2021_tables.zip/fy2021_table16/fy2021_table16 uploaded sucessfully
1


SynapseWidget(Synapse.DataFrame, 2a1a3b16-54a5-4aab-a6d2-0ef5d5d526c0)

...

wasbs://silver@usafactssilver.blob.core.windows.net/immigration_yearbook_excel_files/2021/2022_0920_plcy_refugees_and_asylees_fy2021_tables.zip/fy2021_table15d/fy2021_table15d uploaded sucessfully
1


SynapseWidget(Synapse.DataFrame, 53b090ad-2255-43d4-bb13-bd657ae9f82b)

wasbs://silver@usafactssilver.blob.core.windows.net/immigration_yearbook_excel_files/2021/2022_0920_plcy_refugees_and_asylees_fy2021_tables.zip/fy2021_table19d/fy2021_table19d uploaded sucessfully
1


SynapseWidget(Synapse.DataFrame, 1cd3fccf-8f31-4251-80e9-a41133668c94)

wasbs://silver@usafactssilver.blob.core.windows.net/immigration_yearbook_excel_files/2021/2022_0920_plcy_refugees_and_asylees_fy2021_tables.zip/fy2021_table18d/fy2021_table18d uploaded sucessfully


In [ ]:
if len(bad_records)>= 1:
    pandas_df = pd.DataFrame(bad_records,columns=["URL","File_name","Reason"])
    bad_path = blob_relative_path+'bad_records/bad_record.csv'
    blob_client = container_client.get_blob_client(bad_path)
    csv_file = pandas_df.to_csv(index=False)
    blob_client.upload_blob(csv_file,overwrite=True)

StatementMeta(, , , Cancelled, )